###📌 Introduction: Exploring Customer Behavior in the H&M Fashion Dataset

This project analyzes customer purchasing patterns using a curated sample of the H&M Personalized Fashion Recommendations dataset from Kaggle.
Because the full dataset contains tens of millions of transactions, I created three smaller, representative samples using a Kaggle notebook to ensure reproducibility and fast execution:

sample_transactions.csv — 80,000 transactions

sample_customers.csv — 30,000 customers

sample_articles.csv — all available articles

These samples preserve the relational structure and diversity of the original dataset, making them sufficiently rich for exploratory and descriptive analysis.

In [ ]:
import pandas as pd

transactions = pd.read_csv("https://raw.githubusercontent.com/sirra223/project3512/refs/heads/main/transactions_sample.csv")
customers    = pd.read_csv("https://raw.githubusercontent.com/sirra223/project3512/refs/heads/main/customers_sample.csv")
articles     = pd.read_csv("https://raw.githubusercontent.com/sirra223/project3512/refs/heads/main/articles_sample.csv")

articles.head()
customers.head()
transactions.head()

### 📌 Original H&M EDA Project (Reference Notebook Summary)

The following section reproduces the main parts of the original Kaggle notebook:
https://www.kaggle.com/code/bgourav/eda-h-m-customer-behaviour-and-buying-patterns

I will replicate the author's major analyses using my sample datasets:
- customer demographics
- article attributes and garment groups
- transaction frequency patterns
- price distributions
- category-level analysis




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Make plots a bit prettier
plt.style.use("default")
sns.set_theme(style="whitegrid")

print("Transactions shape:", transactions.shape)
print("Articles shape:", articles.shape)
print("Customers shape:", customers.shape)

display(transactions.head())
display(articles.head())
display(customers.head())


In [ ]:
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

print("\n--- Missing Value Check ---")
print("Missing values in transactions:\n", transactions.isnull().sum())
print("\nMissing values in customers:\n", customers.isnull().sum())
print("\nMissing values in articles:\n", articles.isnull().sum())

# Updated to reassign the result of fillna instead of using inplace=True
customers['club_member_status'] = customers['club_member_status'].fillna('UNKNOWN')
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].fillna('None')

customers['age'] = customers['age'].fillna(customers['age'].median())

In [ ]:
print("\n--- Merging Dataframes ---")
merged_df = pd.merge(transactions, customers, on='customer_id', how='left')
merged_df = pd.merge(merged_df, articles, on='article_id', how='left')

print("Shape of the merged dataframe:", merged_df.shape)
print("First 5 rows of the merged dataframe:")
print(merged_df.head())
print("Column information of the merged dataframe:")
merged_df.info()

# Create new features
merged_df['sales_month'] = merged_df['t_dat'].dt.to_period('M')
merged_df['sales_dayofweek'] = merged_df['t_dat'].dt.day_name()

###📌 Limitations & Notes

Sample Size Reduction:
The full H&M dataset contains tens of millions of rows. For performance, reproducibility, and GitHub compatibility, this project uses a smaller but representative sample (80k transactions, 30k customers). While this reduces volume, it preserves core behavioral patterns.

Missing Demographic Data:
Many customers in the original dataset lack age information. For this project, incomplete ages were cleaned, binned into age ranges, and used only when available.

Price Scaling:
H&M stores prices as scaled decimal values. For interpretability, prices were converted to realistic scales (SEK-based), but they may not exactly represent retail USD values.


Despite these limitations, the dataset remains robust enough to reveal consistent patterns in product preferences, spending behavior, repeat purchases, and color choices.

In [ ]:
print("\n--- EDA on Customer Behavior & Buying Patterns ---")

print("\n--- Overall Sales Trend ---")
sales_by_date = merged_df.groupby('t_dat')['price'].sum()
plt.figure(figsize=(15, 6))
sales_by_date.plot(title='Total Sales Over Time', color='skyblue')
plt.xlabel('Date')
plt.ylabel('Total Sales (in millions)')
plt.show()

In [ ]:
print("\n--- Top 10 Bestselling Products ---")
top_articles = merged_df['article_id'].value_counts().head(10)
top_articles_info = pd.merge(top_articles, articles[['article_id', 'prod_name']], on='article_id')
print(top_articles_info)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_articles_info['count'], y=top_articles_info['prod_name'], palette='viridis')
plt.title('Top 10 Bestselling Products')
plt.xlabel('Number of Transactions')
plt.ylabel('Product Name')
plt.show()

In [ ]:
print("\n--- Top 10 Bestselling Product Groups ---")
top_prod_groups = merged_df['product_group_name'].value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_prod_groups.values, y=top_prod_groups.index, palette='magma')
plt.title('Top 10 Bestselling Product Groups')
plt.xlabel('Number of Transactions')
plt.ylabel('Product Group')
plt.show()

In [ ]:
print("\n--- Customer Segmentation: RFM Analysis ---")

latest_date = transactions['t_dat'].max()
rfm_df = transactions.groupby('customer_id').agg(
    Recency=('t_dat', lambda x: (latest_date - x.max()).days),
    Frequency=('t_dat', 'count'),
    Monetary=('price', 'sum')
)
print("RFM DataFrame Head:")
print(rfm_df.head())

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.histplot(rfm_df['Recency'], bins=50, kde=True, ax=axes[0])
axes[0].set_title('Recency Distribution')
sns.histplot(rfm_df['Frequency'], bins=50, kde=True, ax=axes[1])
axes[1].set_title('Frequency Distribution')
sns.histplot(rfm_df['Monetary'], bins=50, kde=True, ax=axes[2])
axes[2].set_title('Monetary Distribution')
plt.tight_layout()
plt.show()

In [ ]:
print("\n--- Age and Spending Behavior ---")
plt.figure(figsize=(10, 6))
sns.histplot(customers['age'], bins=50, kde=True, color='purple')
plt.title('Distribution of Customer Ages')
plt.xlabel('Age')
plt.ylabel('Number of Customers')
plt.show()

In [ ]:
bins = [0, 18, 25, 35, 45, 55, 65, 100]
labels = ['<18', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']
merged_df['age_group'] = pd.cut(merged_df['age'], bins=bins, labels=labels, right=False)

avg_spending_by_age = merged_df.groupby('age_group')['price'].mean().sort_index()
plt.figure(figsize=(12, 6))
sns.barplot(x=avg_spending_by_age.index, y=avg_spending_by_age.values, palette='coolwarm')
plt.title('Average Transaction Price by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Average Price')
plt.show()

In [ ]:
print("\n--- Seasonality and Day of the Week Analysis ---")
# Sales by month
sales_by_month = merged_df.groupby('sales_month')['price'].sum()
plt.figure(figsize=(15, 6))
sales_by_month.plot(kind='bar', title='Total Sales by Month', color='teal')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.show()

In [ ]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sales_by_day = merged_df.groupby('sales_dayofweek')['price'].sum().reindex(day_order)
plt.figure(figsize=(10, 6))
sns.barplot(x=sales_by_day.index, y=sales_by_day.values, palette='pastel')
plt.title('Total Sales by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
print("\n--- Impact of Fashion News & Club Membership ---")
spending_by_news = merged_df.groupby('fashion_news_frequency')['price'].mean().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=spending_by_news.index, y=spending_by_news.values, palette='dark')
plt.title('Average Transaction Price by Fashion News Frequency')
plt.xlabel('Fashion News Frequency')
plt.ylabel('Average Price')
plt.show()

In [ ]:
spending_by_club = merged_df.groupby('club_member_status')['price'].mean().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=spending_by_club.index, y=spending_by_club.values, palette='cubehelix')
plt.title('Average Transaction Price by Club Member Status')
plt.xlabel('Club Member Status')
plt.ylabel('Average Price')
plt.show()

In [ ]:
print("\n--- Summary of Key Findings ---")
print("Based on the EDA, we can draw the following insights:")
print("- The total sales show a clear trend over time, likely with seasonal peaks.")
print("- Bestselling products and product groups can be identified, guiding inventory and marketing efforts.")
print("- RFM analysis helps segment customers into different tiers (e.g., high-value, recent, frequent).")
print("- Customer age has a visible impact on average spending, which can inform targeted advertising.")
print("- Certain days of the week or months show higher sales, useful for staffing and promotion planning.")
print("- Fashion news subscription and club membership appear to be correlated with customer spending, suggesting these are effective engagement tools.")

### 📌 Additional Analyses (My Contributions)
The original notebook focuses mainly on high-level distributions (who buys, what sells, and when purchases occur).  
In the following section, I extend this work by examining more detailed relationships between customers and products, such as:

1. **Age × Category Preference Analysis**  
   - Identifies which product categories different customer age groups purchase most often.

2. **Price Sensitivity by Age Group**  
   - Examines how average purchase price varies across age segments and product types.

3. **Category-Level Repeat Purchase Behavior**  
   - Measures how frequently customers buy from the same category more than once, revealing category “loyalty” or repeat interest.

4. **Color Preference by Category**  
   - Analyzes which color groups dominate each product group and how color choices relate to customer style and product type.



### Analysis #1 - Age × Category Preference

In this analysis, I examine how product category preferences differ across age groups.  
Using the merged transactions table, and picking the top 10 categories.

This reveals which categories are relatively more popular among younger vs. older customers and helps H&M understand generational style preferences.


**“Do younger and older shoppers buy different types of clothing, or are their preferences the same?”**

In [ ]:
sns.set_theme(style="whitegrid")
plt.style.use("default")

# Merge
tx_full = transactions.merge(
    articles,
    on='article_id',
    how='left',
    validate='many_to_one'
).merge(
    customers,
    on='customer_id',
    how='left',
    validate='many_to_one'
)

tx_full["price_scaled"] = tx_full["price"] * 1000
SEK_to_USD = 0.10

tx_full["price_dollar"] = tx_full["price_scaled"] * SEK_to_USD



tx_full['age'] = pd.to_numeric(tx_full['age'], errors='coerce')

tx_full = tx_full[(tx_full['age'] >= 10) & (tx_full['age'] <= 100)]


age_bins = [10, 20, 30, 40, 50, 60, 120]
age_labels = ["10–19", "20–29", "30–39", "40–49", "50–59", "60+"]

tx_full['age_group'] = pd.cut(
    tx_full['age'],
    bins=age_bins,
    labels=age_labels,
    right=False
)


tx_full[['age', 'age_group']].head()

In [ ]:
print("Age group distribution:")
tx_full['age_group'].value_counts().sort_index()


In [ ]:

age_cat = tx_full.dropna(subset=['age_group', 'product_group_name'])

# Count transactions by age group and product group
age_cat_counts = (
    age_cat.groupby(['age_group', 'product_group_name'])['article_id']
    .count()
    .reset_index(name='txn_count')
)

age_totals = age_cat_counts.groupby('age_group')['txn_count'].transform('sum')
age_cat_counts['percent'] = age_cat_counts['txn_count'] / age_totals * 100


In [ ]:
age_cat_pivot = age_cat_counts.pivot(
    index='age_group',
    columns='product_group_name',
    values='percent'
)

# Ensure all age groups appear
age_cat_pivot = age_cat_pivot.reindex(age_labels)

# Replace missing with 0
age_cat_pivot = age_cat_pivot.fillna(0)


In [ ]:
col_totals = age_cat_pivot.sum(axis=0)
top_cats = col_totals.sort_values(ascending=False).head(10).index

age_cat_pivot_top = age_cat_pivot[top_cats]


In [ ]:
from google.colab import files

plt.figure(figsize=(12, 6))
sns.heatmap(
    age_cat_pivot_top,
    annot=True,
    fmt=".1f",
    cmap="Reds"
)
plt.title("Age Group × Product Group Preference (% of Transactions)")
plt.xlabel("Product Group")
plt.ylabel("Age Group")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

age_cat_pivot_top

plt.savefig('analysis1heatmap.png')

files.download('analysis1heatmap.png')

Older age groups (particularly 60+) have the highest share of purchases in Garment Upper Body and Garment Lower Body.

Younger groups (10–19 and 20–29) over-index slightly in categories like Swimwear and Underwear.

Across all ages, upper-body garments dominate, meaning tops are the most purchased category regardless of age.

In [ ]:
top_age_per_cat = (
    age_cat_pivot_top
    .idxmax(axis=0)
    .reset_index()
)

top_age_per_cat.columns = ["product_group_name", "top_age_group"]

top_percent_per_cat = (
    age_cat_pivot_top
    .max(axis=0)
    .reset_index()
)

top_percent_per_cat.columns = ["product_group_name", "top_percent"]

top_cat_summary = top_age_per_cat.merge(top_percent_per_cat, on="product_group_name")

top_cat_summary


In [ ]:
palette = {
    "10–19": "#fde0dd",
    "20–29": "#fcc5c0",
    "30–39": "#fa9fb5",
    "40–49": "#dd3497",
    "50–59": "##f768a1",
    "60+":   "#ae017e"
}


plt.figure(figsize=(10,6))
sns.barplot(
    data=top_cat_summary,
    x="product_group_name",
    y="top_percent",
    hue="top_age_group",
    palette=palette
)

plt.title("Which Age Group Buys Each Category the Most?")
plt.xlabel("Product Category")
plt.ylabel("Top Purchase Percentage")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.savefig('analysis1bar.png')

files.download('analysis1bar.png')

- Customers **aged 50–60+** make the highest share of purchases in classic basics such as *garment upper body* and *garment lower body*, showing a preference for simpler, everyday clothing. - **Nightwear**, although representing only ~1% of total purchases, is also most commonly purchased by older customers (50–60+). This suggests that comfort-oriented homewear is more popular among older age groups.
- Customers **aged 10–19** dominate categories like *underwear*, *swimwear*, and *accessories*, which are more trend-driven and seasonally influenced.
- The **30–39** age group leads in *garment full body* and *shoes*, suggesting a stronger focus on complete outfits and wardrobe staples. Socks & Tights although in lower-volume are most popular among customers aged **30–39 and 40–49**, indicating that mid-aged shoppers tend to purchase more functional and replacement accessories than younger shoppers.



Overall, the data rather small or large reveals clear generational style patterns: older customers prefer basics, younger customers gravitate toward expressive/trendy categories, middle-age customers buy more structured and functional items.


--------------------------------------------------------------------------
## Analysis #2 - Price Sensitivity by Age

How does the *average purchase price* vary across age groups and product categories?
Understanding this helps us see which age segments are more price-sensitive and which
are willing to spend more on certain types of products.

**Methods**

- Use the cleaned `age` and binned `age_group` (10–19, 20–29, 30–39, 40–49, 50–59, 60+).
- Focus on **price per transaction line** in the sampled dataset.
- Compare:
  - Overall price distribution by age group
  - Average price by age group **within major product categories**.


**How does average purchase price vary across age segments and product types?**

In [ ]:
df = tx_full.copy()

df["age"] = pd.to_numeric(df["age"], errors="coerce")
df["price_dollar"] = pd.to_numeric(df["price_dollar"], errors="coerce")



df = df[(df["age"] >= 10) & (df["age"] <= 100)]
df = df.dropna(subset=["age", "price"])

# Define age bands
age_bins = [10, 20, 30, 40, 50, 60, 200]
age_labels = ["10-19", "20-29", "30-39", "40-49", "50-59", "60+"]

df["age_band"] = pd.cut(
    df["age"],
    bins=age_bins,
    labels=age_labels,
    right=False
)


print(df["age_band"].value_counts(dropna=False))


In [ ]:
tx_price = df.copy()

price_summary_age = (
    tx_price
    .groupby("age_band")["price_dollar"]
    .agg(["count", "mean", "median", "min", "max"])
    .reset_index()
    .rename(columns={
        "age_band": "age_group",
        "count": "n_transactions",
        "mean": "avg_price",
        "median": "median_price"
    })
)

price_summary_age


In [ ]:
age_order = ["10-19", "20-29", "30-39", "40-49", "50-59", "60+"]

tx_price["age_band"] = tx_price["age_band"].cat.reorder_categories(
    age_order, ordered=True
)


In [ ]:
import numpy as np
# Log transform because H&M prices are heavily right-skewed
df["price_log"] = np.log1p(df["price_dollar"])

plt.figure(figsize=(10,5))
sns.boxplot(
    data=df,
    x="age_band",
    y="price_log",
    order=["10-19","20-29","30-39","40-49","50-59","60+"],
    palette="Set2"
)
plt.xlabel("Age Group")
plt.ylabel("log(1 + price)")
plt.title("Distribution of Transaction Prices by Age Group")
plt.show()


The boxplot compares the distribution of transaction-level prices across all age segments using a log scale. Overall, price levels are extremely consistent between age groups, reflecting H&M’s standardized pricing strategy.

The **10–19** and **20–29** age groups show the lowest median prices, indicating slightly higher price sensitivity among younger shoppers.

From **30–59**, the median price increases marginally, and these groups show more high-price outliers, suggesting that older customers occasionally spend more.

The **60+** segment sits between the younger and mid-aged groups, with a modest number of premium purchases.

Across all ages, the middle 50% of purchases fall within a very tight range, confirming that age has a limited impact on transaction-level pricing.

Overall, this chart shows that age-based price differences are subtle, and H&M customers across all age groups generally purchase items in similar price ranges. The category of the product (e.g., Shoes vs. Underwear) matters more than the age of the shopper.

In [ ]:
top_cats = tx_full["product_group_name"].value_counts().head(8).index

temp = tx_full[tx_full["product_group_name"].isin(top_cats)]


heat_data = (
    temp.groupby(["product_group_name", "age_group"])["price_dollar"]
        .mean()
        .reset_index()
        .pivot(index="product_group_name", columns="age_group", values="price_dollar")
)

plt.figure(figsize=(10,6))
sns.heatmap(
    heat_data,
    annot=True,
    fmt=".2f",
    cmap="YlGnBu"
)
plt.title("Average Price by Product Group and Age Group")
plt.xlabel("Age Group")
plt.ylabel("Product Group")
plt.show()

H&M’s price column (price) in the dataset is stored as a decimal fraction of the real price. To evaluate how price varies across customer ages and product categories, I calculated the average item price for each Age Group × Product Group combination.
The heatmap shows that price differences across age groups are small, indicating that H&M shoppers tend to purchase items within similar price ranges regardless of age.
However, price varies strongly across product types. Shoes and full-body garments consistently have the highest average prices, while underwear, socks/tights, and accessories are the lowest.
This suggests that age has a minor impact on price sensitivity, whereas product category plays a much larger role in determining the price customers pay.

In [ ]:
import seaborn as sns
sns.set(style="white")

plt.figure(figsize=(10,7))
sns.violinplot(
    data=df,
    x="price",
    y="age_band",
    scale="width",
    inner="quartile",
    palette="Spectral"
)
plt.title("Price Distribution by Age Group (Ridgeline-Style Violin Plot)")
plt.show()


Note: The H&M dataset stores prices as unit prices (e.g., 0.05 ≈ $5). Therefore, the values in the violin plot represent scaled prices rather than whole-dollar amounts. The shape of the price distributions is almost identical across all ages, further proving that price sensitivity is extremely stable across demographics.

Younger groups **(10–19, 20–29)** show slightly more purchases near the lower-price range, but the difference is visually small.

Older groups have slightly smoother distributions, suggesting more consistent purchasing behavior, but still within the same price band.

Overall, Price sensitivity shows almost no meaningful variation by age.
Instead, product category is the primary driver of price differences. All age groups purchase items within the same narrow price range.

The differences between medians across ages are statistically small.
Categories like Shoes and Garment Full Body consistently cost more, regardless of who buys them.


--------------------------------------------------------------------------
##Analysis #3 - Repeat Customer behavior


### Objective
To understand customer loyalty at the product category level, I examined which categories customers return to most frequently. A repeat customer is defined as someone who purchased from the same product category at least twice during the sample period.

### Method
- Aggregated transactions by customer and product group.
- Counted the number of purchases per customer-category pair.
- Defined `is_repeat = True` when purchases ≥ 2.
- Calculated repeat rate for each product group:

**Which product categories do customers come back to the most, and how does repeat behavior differ across age groups?**

In [ ]:
cust_cat = (
    tx_full
    .dropna(subset=["customer_id", "product_group_name"])
    .groupby(["customer_id", "product_group_name"])
    .size()
    .reset_index(name="n_purchases")
)


In [ ]:
cust_cat["is_repeat"] = cust_cat["n_purchases"] >= 2
cust_cat.head()


In [ ]:
category_loyalty = (
    cust_cat.groupby("product_group_name")
    .agg(
        total_customers=("customer_id", "nunique"),
        repeat_customers=("is_repeat", "sum")
    )
    .reset_index()
)

category_loyalty["repeat_rate"] = (
    category_loyalty["repeat_customers"] / category_loyalty["total_customers"]
)

category_loyalty


In [ ]:
top_repeat = (
    category_loyalty
    .sort_values("repeat_rate", ascending=False)
    .head(10)
)

plt.figure(figsize=(10,5))
sns.barplot(
    data=top_repeat,
    x="repeat_rate",
    y="product_group_name",
    palette="Blues_r"
)
plt.xlabel("Repeat Rate (Share of Customers with ≥2 Purchases)")
plt.ylabel("Product Group")
plt.title("Top Categories by Repeat Customer Behavior")
plt.xlim(0, 1)
plt.show()


The categories with the strongest repeat behavior were:
- **Garment Upper Body**
- **Garment Lower Body**
- **Swimwear**
- **Garment Full Body**

These categories reflect wardrobe staples and style-based items that customers buy frequently.

Moderate Repeat Categories
- **Underwear**
- **Accessories**

These show stable but not dominant repeat behavior.

Lowest Repeat Categories
- **Shoes**
- **Socks & Tights**
- **Nightwear**

These appear to be purchased infrequently or as one-off needs during the sample window.

In [ ]:
loyalty_age = (
    cust_cat.merge(tx_full[["customer_id", "age_group"]].drop_duplicates(),
                   on="customer_id", how="left")
    .groupby(["product_group_name", "age_group"])
    .agg(
        total_customers=("customer_id", "nunique"),
        repeat_customers=("is_repeat", "sum")
    )
    .reset_index()
)

loyalty_age["repeat_rate"] = (
    loyalty_age["repeat_customers"] / loyalty_age["total_customers"]
)


In [ ]:
repeat_age_df = loyalty_age.copy()


valid_cats = category_loyalty[category_loyalty['repeat_rate'] > 0]["product_group_name"]
repeat_age_df = repeat_age_df[repeat_age_df["product_group_name"].isin(valid_cats)]


heat = repeat_age_df.pivot(index="product_group_name", columns="age_group", values="repeat_rate")

sorted_product_groups = category_loyalty.sort_values("repeat_rate", ascending=False)["product_group_name"]
heat = heat.reindex(sorted_product_groups).dropna(how='all')

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(heat, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Repeat Category Behavior by Age Group")
plt.xlabel("Age Group")
plt.ylabel("Product Group")
plt.show()

The heatmap shows that:
- **Repeat behavior is consistently highest in ages 20–49**, across most categories.
- **Garment Upper Body** is the strongest loyalty category for every age group.
- **Underwear** has stronger repeat behavior among customers aged 30–49.
- **Shoes** show increasing repeat rates with age.
- **Nightwear** shows weak repeat behavior across all ages.

**Conclusion**

Overall, the plot reinforces that repeat purchasing is more driven by the type of product (e.g., essential garments) than by the age of the customer. However, age does influence loyalty within certain categories, highlighting opportunities for targeted marketing.
Wardrobe essentials show the strongest loyalty patterns, while seasonal or one-time categories show the weakest. This suggests product-level buying habits are more stable than age-based buying habits. Repeat category behavior at H&M is shaped primarily by the function of the product, not by who buys it.


------------------------------------------------------
## Analysis #4 - Color Preference by Product Group and Age

### Objective
This analysis explores how color influences purchasing behavior in the H&M dataset.  
I first identify which colors are most popular overall, then examine whether color preferences vary by product category or customer age group.  
Understanding color preference offers insight into customer aesthetics, merchandising patterns, and core brand palette usage.

---

### Method
- Filtered transactions to keep valid `product_group_name` and `colour_group_name` entries.  
- Counted total purchases per color to measure overall popularity.   
- Focused on **Garment Upper Body**, one of the most frequently purchased categories, to examine color preference by age group.  
- Calculated the share of each top color within each age band to compare color preferences across demographics.





**Which colors truly define customer behavior and does age change the story?**

In [ ]:
!pip install squarify


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import squarify  # for treemaps

sns.set(style="whitegrid")

# Keep only valid rows
df_color = tx_full.dropna(subset=["product_group_name", "colour_group_name"]).copy()

# OPTIONAL: Clean color names (strip spaces, uppercase first letter)
df_color["colour_group_name"] = df_color["colour_group_name"].str.strip()


In [ ]:
import squarify
import matplotlib.pyplot as plt

# Get top 15 most common colors
top_colors = (
    df_color["colour_group_name"]
    .value_counts()
    .head(15)
    .reset_index()
)
top_colors.columns = ["colour_group_name", "count"]

plt.figure(figsize=(12, 8))
squarify.plot(
    sizes=top_colors["count"],
    label=top_colors["colour_group_name"],
    alpha=0.85,
    color=sns.color_palette("Spectral", len(top_colors))
)
plt.title("Top 15 Most Popular Colors (Treemap)")
plt.axis("off")
plt.show()


The treemap visualizes the **Top 15 most popular colors** in the dataset. Each block represents the relative share of purchases attributed to each color. Neutral tones dominate H&M’s product mix. **Black** and **White** occupy the largest portions of the treemap, indicating heavy customer preference as well as strong representation in the product catalog.  
Additional neutrals such as **Beige, Light Beige, Off White, Grey, and Dark Blue** also appear prominently. Brighter colors (Red, Pink, Light Blue, Greenish Khaki) exist but represent smaller shares.
This reflects H&M’s core aesthetic: versatile basics and foundational wardrobe colors.

In [ ]:
color_counts = df_color["colour_group_name"].value_counts().head(10)

plt.figure(figsize=(8,8))
plt.pie(
    color_counts,
    labels=color_counts.index,
    autopct='%1.1f%%',
    startangle=90,
    pctdistance=0.85,
    colors=sns.color_palette("Spectral", len(color_counts))
)

# Draw circle for donut effect
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.title("Top 10 Colors (Donut Chart)")
plt.tight_layout()
plt.show()


The donut chart quantifies the relative percentages of the top colors. Revealing the dominance of neutrals numerically. We can see that **Black alone accounts for nearly half of all color purchases**, followed by White.  Collectively, neutral colors make up the majority of the dataset. Saturated or vibrant colors represent a smaller proportion, suggesting they are used more selectively or appear in specific seasonal collections.


In [ ]:
# Count purchases by category + color
cat_color = (
    df_color.groupby(["product_group_name", "colour_group_name"])
    .size()
    .reset_index(name="count")
)

# Compute share of colors within each category
cat_color["share"] = (
    cat_color["count"] /
    cat_color.groupby("product_group_name")["count"].transform("sum")
)

# Keep top 5 colors per category
top5_colors = (
    cat_color.sort_values(["product_group_name", "share"], ascending=False)
    .groupby("product_group_name")
    .head(5)
)


In [ ]:
color_age = (
    df_color.dropna(subset=["age_group"])
    .groupby(["product_group_name", "age_group", "colour_group_name"])
    .size()
    .reset_index(name="count")
)

# Share of colors within each Age × Category
color_age["share"] = (
    color_age["count"] /
    color_age.groupby(["product_group_name", "age_group"])["count"].transform("sum")
)


In [ ]:
category_to_plot = "Garment Upper body"

# Top 5 colors overall in this category
top_colors_cat = (
    cat_color[cat_color["product_group_name"] == category_to_plot]
    .sort_values("share", ascending=False)["colour_group_name"]
    .head(5)
)

# Filter only those colors
df_plot = color_age[
    (color_age["product_group_name"] == category_to_plot) &
    (color_age["colour_group_name"].isin(top_colors_cat))
]

# Pivot for stacked bar
pivot_cat = df_plot.pivot(
    index="age_group", columns="colour_group_name", values="share"
).fillna(0)


**Does color preference vary by age?**

In [ ]:
pivot_cat.plot(
    kind="bar", stacked=True, figsize=(10,5),
    colormap="Spectral"
)
plt.title(f"Color Preference by Age Group — {category_to_plot}")
plt.xlabel("Age Group")
plt.ylabel("Share of Purchases")
plt.legend(title="Color", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


To examine whether color preference varies across age demographics, I analyzed the top five colors within the **Garment Upper Body** product group and calculated the share of purchases contributed by each age band.

Across all age groups, color preference remains relatively consistent. **Black** remains the most purchased color in every age band. **White, Dark Blue, Light Beige, and Beige** follow similar patterns with slight variation. We can see that younger shoppers **(10–19)** show slightly higher shares of lighter or pastel tones. While middle-age groups **(20–49)** display the most balanced distribution across the top colors. Older customers **(50–59, 60+)** maintain strong preference for classic neutrals.

These results indicate that **color preference at H&M is more product-driven than age-driven**.  
Age plays a minor role, but overall customers share a similar color palette preference.

In conclusion, Color preference in the H&M dataset is overwhelmingly dominated by neutral tones, with **Black and White** leading by a wide margin. While vibrant colors are present, they represent a small fraction of total purchases.  When segmented by age within a key category (Garment Upper Body), the patterns remain highly consistent across demographics.  This suggests that color preference at H&M is shaped primarily by **merchandising and product assortment**, with customer age having only subtle influence.

---
### 📌 Final Summary

Across all four analyses, several consistent patterns emerged:

Age affects what customers buy more than how much they spend. Older shoppers **(40–60+)** lean towards high-volume categories like Garment Upper Body, but price distributions remain nearly identical across age groups.

Product category is the strongest driver of behavior.
Shoes and full-body garments consistently carry higher price points, regardless of who buys them.
Upper Body items show the strongest repeat engagement across all ages.

Color preferences are remarkably stable across the customer base.
Black overwhelmingly dominates across all categories and age groups, followed by White and neutral tones. Only a few categories, such as Swimwear, show meaningful bursts of color variation.

Repeat purchase behavior aligns with everyday apparel needs. Customers repeatedly return to core clothing categories, while accessories and specialty items show lower repeat rates.

Overall, the analyses reveal a fashion retail landscape driven not by demographic polarization but by universal customer behaviors centered around convenience, versatility, and wardrobe basics.

**Next steps**

Future work could build on these findings by incorporating time-based patterns (seasonality and trend shifts), clustering customers by behavior rather than demographics, and exploring basic recommendation approaches to identify related products. Additional analyses could also examine price elasticity within categories or investigate which items frequently appear together to understand outfit-level purchasing behavior.